# Identify entities:
### Places and Persons using DbPedia

In [5]:
import requests
import pycurl
from urllib.request import urlopen
from urllib.parse import quote
import json
import os
import pandas as pd
from _datetime import date

In [2]:
# 1. give each paragraph an id
# 2. for each paragraph obtain the list of people entities
# 3. check each people found in step 2, if they are type Music
# 4. check identify places

In [3]:
urlAnnotation = 'http://dbpedia-spotlight.en:80/rest/annotate/'
def setDbPediaAnnotationServiceParameters(text):
    """ Se parameters for querying Dbpedia
    args: text - text to be analysed
    return: headers
            params
    """
    headers = {
        'Accept':'application/json',
        "content-type":"application/x-www-form-urlencoded"
    }

    params = {
        'confidence': '0.4',
        "text":text,
    }
    return headers, params

def setDbPediaAnnotationServiceParametersTypes(text,types):
    """ Se parameters for querying Dbpedia
    args: text - text to be analysed
        types: different types of entities
    return: headers
            params
    """
    headers = {
        'Accept':'application/json',
        "content-type":"application/x-www-form-urlencoded"
    }

    params = {
        'types' : types,
        'confidence': '0.35',
        "text":text,
    }
    return headers, params

# return response in JSON format
def queryDBPediaAnnotation(url,header,params):
    response = requests.post(url,headers=header, params=params).json()
    return response

In [4]:
def listFiles (directory):
    patientList = [f for f in os.listdir(directory) if not f.startswith('.')] 
    return patientList
def openTextFile(directory,filename):
    fileObject = open(directory+filename, "r")
#     data = fileObject.read()
    data = fileObject.readlines()
#     print(data)
    return data

In [14]:
# for each paragraph in the item
# people = {'Glenn_Miller':'64610','Andrés_Segovia':'71932','Django_Reinhardt':'9039','Maria_Callas':'64966','Édith_Piaf':'64963','John_Lennon':'15852'}
# people = {'64610':'Glenn_Miller','71932':'Andrés_Segovia','9039':'Django_Reinhardt','64966':'Maria_Callas','64963':'Édith_Piaf','15852':'John_Lennon'}
people_list = {'9039':'Django_Reinhardt'}

# files = listFiles('cleanText/')
dfsummary = pd.DataFrame()
for file in people_list:
    print(people_list[file])
    # open file
    biography = openTextFile('cleanText/',file+'.txt')
    # start counter for paragraph identifier
    prgId = 0
    dfparagraphs = pd.DataFrame()
    
    for prg in biography:
#         print(prg)
        prgId += 1
        ## send text and return params
        hdrs, prms = setDbPediaAnnotationServiceParameters(prg)
        responseJSON = queryDBPediaAnnotation(urlAnnotation,hdrs,prms)
#         print(responseJSON)
        
        if 'Resources' in responseJSON:
            df = pd.DataFrame.from_dict(responseJSON['Resources'])
#         if len(df)>0:
            df.sort_values(by=['@similarityScore'],inplace=True,ascending=[False])
            df.drop_duplicates(subset = ['@URI','@similarityScore','@surfaceForm'],keep = "first", inplace = True)
        df['paragraph']=prg
        dfparagraphs = dfparagraphs.append(df)
#         dfparagraphs.info()
        
    dfparagraphs['wikiPageID'] = file.replace('.txt','')
    dfparagraphs['artistName'] = people_list[file.replace('.txt','')]
    dfsummary = dfsummary.append(dfparagraphs)

Django_Reinhardt
<class 'pandas.core.frame.DataFrame'>
Int64Index: 17 entries, 18 to 11
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   @URI                     17 non-null     object
 1   @support                 17 non-null     object
 2   @types                   17 non-null     object
 3   @surfaceForm             17 non-null     object
 4   @offset                  17 non-null     object
 5   @similarityScore         17 non-null     object
 6   @percentageOfSecondRank  17 non-null     object
 7   paragraph                17 non-null     object
dtypes: object(8)
memory usage: 1.2+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 29 entries, 18 to 3
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   @URI                     29 non-null     object
 1   @support                 29 non-null     obj

In [7]:
dfsummary.info()
dfsummary.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 848 entries, 14 to 2
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   @URI                     848 non-null    object
 1   @support                 848 non-null    object
 2   @types                   848 non-null    object
 3   @surfaceForm             848 non-null    object
 4   @offset                  848 non-null    object
 5   @similarityScore         848 non-null    object
 6   @percentageOfSecondRank  848 non-null    object
 7   paragraph                848 non-null    object
 8   wikiPageID               848 non-null    object
 9   artistName               848 non-null    object
dtypes: object(10)
memory usage: 72.9+ KB


,@URI,@support,@types,@surfaceForm,@offset,@similarityScore,@percentageOfSecondRank,paragraph,wikiPageID,artistName
14,http://dbpedia.org/resource/American_Patrol,29,"Wikidata:Q386724,Wikidata:Q2188189,Schema:Musi...",American Patrol,440,1.0,0.0,"Alton Glenn Miller (March 1, 1904 – disappeare...",64610,Glenn_Miller
10,http://dbpedia.org/resource/Moonlight_Serenade,106,"Wikidata:Q386724,Wikidata:Q2188189,Schema:Musi...",Moonlight Serenade,304,1.0,9.764469070529064E-55,"Alton Glenn Miller (March 1, 1904 – disappeare...",64610,Glenn_Miller
27,http://dbpedia.org/resource/English_Channel,7343,"Schema:Place,Schema:BodyOfWater,DBpedia:Place,...",English Channel,937,1.0,5.021093162121786E-22,"Alton Glenn Miller (March 1, 1904 – disappeare...",64610,Glenn_Miller
19,http://dbpedia.org/resource/Elvis_Presley,10377,"Http://xmlns.com/foaf/0.1/Person,Wikidata:Q5,W...",Elvis Presley,625,1.0,7.21480869762467E-26,"Alton Glenn Miller (March 1, 1904 – disappeare...",64610,Glenn_Miller
17,http://dbpedia.org/resource/Anvil_Chorus,38,,Anvil Chorus,517,1.0,0.0,"Alton Glenn Miller (March 1, 1904 – disappeare...",64610,Glenn_Miller


In [15]:
dfsummary.to_csv('datasets/'+str(date.today())+'_9039_entities_filter.csv',index=False)

In [16]:
entitiyTypes = 'DBpedia:Person,DBPedia:Place,dbo:MusicalArtist'
# for each paragraph in the item
# people = {'Glenn_Miller':'64610','Andrés_Segovia':'71932','Django_Reinhardt':'9039','Maria_Callas':'64966','Édith_Piaf':'64963','John_Lennon':'15852'}
# people_list = {'64610':'Glenn_Miller','71932':'Andrés_Segovia','9039':'Django_Reinhardt','64966':'Maria_Callas','64963':'Édith_Piaf','15852':'John_Lennon'}
people_list = {'9039':'Django_Reinhardt'}

# files = listFiles('cleanText/')
dfsummary = pd.DataFrame()
for file in people_list:
#     print(people_list[file.replace('.txt','')])
    # open file
    biography = openTextFile('cleanText/',file+'.txt')
    # start counter for paragraph identifier
    prgId = 0
    dfparagraphs = pd.DataFrame()
    for prg in biography:
#         print(prg)
        prgId += 1
        ## send text and return params
        hdrs, prms = setDbPediaAnnotationServiceParametersTypes(prg,entitiyTypes)
        responseJSON = queryDBPediaAnnotation(urlAnnotation,hdrs,prms)
#         print(responseJSON)
        
        if 'Resources' in responseJSON:
            df = pd.DataFrame.from_dict(responseJSON['Resources'])
#         if len(df)>0:
            df.sort_values(by=['@similarityScore'],inplace=True,ascending=[False])
            df.drop_duplicates(subset = ['@URI','@similarityScore','@surfaceForm'],keep = "first", inplace = True)
        df['paragraph']=prg
        dfparagraphs = dfparagraphs.append(df)
#         dfparagraphs.info()
        
    dfparagraphs['wikiPageID'] = file.replace('.txt','')
    dfparagraphs['artistName'] = people_list[file.replace('.txt','')]
    dfsummary = dfsummary.append(dfparagraphs)

Jean Reinhardt (23 January 1910 – 16 May 1953), known to all by his Romani nickname Django (French: [dʒãŋɡo ʁɛjnaʁt] or [dʒɑ̃ɡo ʁenɑʁt]), was a Belgian-born Romani-French jazz guitarist and composer. He was the first major jazz talent to emerge from Europe and remains the most significant.With violinist Stéphane Grappelli, Reinhardt formed the Paris-based Quintette du Hot Club de France in 1934. The group was among the first to play jazz that featured the guitar as a lead instrument. Reinhardt recorded in France with many visiting American musicians, including Coleman Hawkins and Benny Carter, and briefly toured the United States with Duke Ellington's orchestra in 1946. He died suddenly of a stroke in 1953 at the age of 43.

{'@text': "Jean Reinhardt (23 January 1910 â\x80\x93 16 May 1953), known to all by his Romani nickname Django (French: [dÊ\x92Ã£Å\x8bÉ¡o Ê\x81É\x9bjnaÊ\x81t] or [dÊ\x92É\x91Ì\x83É¡o Ê\x81enÉ\x91Ê\x81t]), was a Belgian-born Romani-French jazz guitarist and composer.

In [17]:
dfsummary.to_csv('datasets/'+str(date.today())+'_9039_entities_filterTypes.csv',index=False)